In [1]:
import pandas as pd
from IPython.display import Markdown, display
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np
from sklearn import metrics
import StochasticGhost

def printmd(string):
    display(Markdown(string))
        
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from sklearn.preprocessing import StandardScaler 
from torch.nn.utils import clip_grad_norm_ 

In [2]:
#adult = pd.read_csv('adult.csv')

column_names = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num','marital-status', 'occupation', 'relationship', 'race', 'gender','capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income']

adult = pd.read_csv('adult.data', names=column_names)

In [3]:
############# If you want to drop the missing value rows ################
mask = adult.eq(' ?')
mask.sum()
adult = adult.drop(adult.index[mask.any(axis=1)]).reset_index(drop=True)
len(adult)

# Setting all the categorical columns to type category
for col in set(adult.columns) - set(adult.describe().columns):
    adult[col] = adult[col].astype('category')


adult['hour_worked_bins'] = ['<40' if i < 40 else '40-60' if i <= 60 else '>60'  for i in adult['hours-per-week']]
adult['hour_worked_bins'] = adult['hour_worked_bins'].astype('category')

In [4]:
# Create one hot encoding of the categorical columns in the data frame.
def oneHotCatVars(df, df_cols):
    
    df_1 = adult_data = df.drop(columns = df_cols, axis = 1)
    df_2 = pd.get_dummies(df[df_cols])
    
    return (pd.concat([df_1, df_2], axis=1, join='inner'))


In [5]:
SENSITIVE_CODE_0 = 0
SENSITIVE_CODE_1 = 1

gender_mapping = {' Male': SENSITIVE_CODE_0, ' Female': SENSITIVE_CODE_1}

adult_new = adult.copy()
print("Unique values in 'gender' column in the new DataFrame:", adult_new['gender'].unique())

Unique values in 'gender' column in the new DataFrame: [' Male', ' Female']
Categories (2, object): [' Female', ' Male']


In [6]:
# Create a new column 'race_code' based on the mapping
adult_new['gender_code'] = adult_new['gender'].map(gender_mapping)

income_mapping = {' >50K': 1, ' <=50K': 0}
adult_new['income_code'] = adult_new['income'].map(income_mapping)

adult_new['education_code'] = pd.Categorical(adult_new['education']).codes
adult_new['marital_code'] = pd.Categorical(adult_new['marital-status']).codes
adult_new['occupation_code'] = pd.Categorical(adult_new['occupation']).codes
adult_new['relationship_code'] = pd.Categorical(adult_new['relationship']).codes
adult_new['race_code'] = pd.Categorical(adult_new['race']).codes
adult_new['country_code'] = pd.Categorical(adult_new['native-country']).codes
adult_new['hours_code'] = pd.Categorical(adult_new['hour_worked_bins']).codes

in_cols = ['education_code', 'marital_code', 'gender_code', 'occupation_code', 'relationship_code', 'race_code', 'country_code', 'hours_code']
out_cols = ['income_code']

In [7]:
GENDER_IND = 2

x_train, x_val, y_train, y_val = train_test_split(adult_new[in_cols].values, adult_new[out_cols].values, test_size  = 0.30)

# Normalization

scaler = StandardScaler()  

# Fitting only on training data
scaler.fit(x_train)  
X_train = scaler.transform(x_train)  

# Applying same transformation to test data
X_val = scaler.transform(x_val)

Raw and scaled data are saved as we need these for further analysis while selecting our final model in the model_selector file.

In [8]:
# Assuming x_val and y_val are numpy arrays
# Convert y_val to a column vector to match the shape of x_val
#y_val = np.expand_dims(y_val, axis=1)

# Concatenate x_val and y_val along the columns
data_combined_raw = np.concatenate((x_val, y_val), axis=1)

# Convert the combined data to a DataFrame
df_combined = pd.DataFrame(data_combined_raw)

x_val_columns = ['education_code', 'marital_code', 'gender_code', 'occupation_code', 'relationship_code', 'race_code', 'country_code', 'hours_code']
y_val_columns = ['income_code']

df_combined.columns = x_val_columns + y_val_columns

df_combined.to_csv('val_data_income/val_data_raw_income.csv', index=False)



data_combined_scaled = np.concatenate((X_val, y_val), axis=1)
df_combined = pd.DataFrame(data_combined_scaled)
df_combined.to_csv('val_data_income/val_data_scaled_income.csv', index=False)

The objective and constraints should be defined here.

For this case the Constraint optimization problem looks like this:

$$\min\hspace{0.1cm} \{ \text{MSE}(y_{\text{pred}}-y_{\text{true}}) \} \\
    \text{s.t. } -\epsilon < \text{MSE}(y_{\text{pred}|S=0} - y_{\text{true}|S=0}) - \text{MSE}(y_{\text{pred}|S=1} - y_{\text{true}|S=1}) < \epsilon$$

In [9]:
class CustomNetwork(nn.Module):

    # For now the input data is passed as init parameters
    def __init__(self, layer_sizes, itrain, otrain, ival, oval, itrain_raw):
        super(CustomNetwork, self).__init__()

        # Create a list of linear layers based on layer_sizes
        self.itrain = itrain
        self.otrain = otrain
        self.ival = ival
        self.oval = oval
        self.itrain_raw = itrain_raw
        self.layers = nn.ModuleList()
        self.layer_norms = nn.ModuleList()
        self.RANDOM_SEED = 0
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = torch.relu((layer(x)))
        x = torch.sigmoid(self.layers[-1](x))
        return x
    

    ######Only this loss function is used here######
    def compute_loss(self, Y, Y_hat):
        L_sum = 0.5*torch.sum(torch.square(Y - Y_hat))

        m = Y.shape[0]
        # print("Y shape is: ", m)
        L = (1./m) * L_sum

        return L

    def bce_loss(self, outputs, targets):
        criterion = nn.BCELoss()
        loss = criterion(outputs, targets)
        if torch.isnan(loss).any():
            for name, param in self.named_parameters():
                print(name)
                print(param.data)
        return loss

    def obj_fun(self, params, minibatch):
        model_parameters = list(self.parameters())
        x = self.itrain
        y = self.otrain
        #x_female = x[:, ]
        samples = np.random.choice(len(y), minibatch, replace=False)
        for i in range(len(params)):
            model_parameters[i].data = torch.Tensor(params[i])

        #print("LOOKIE:",x[samples, :])
        obj_fwd = self.forward(x[samples, :]).flatten()
        if torch.isnan(obj_fwd).any():
            print("THE MINIBATCH SIZE>>>>>",minibatch)
            for name, param in self.named_parameters():
                print(name)
                print(param.data)
        #print("LOOKIE the predicted values: ",obj_fwd)
        #print(obj_fwd.shape)
        #print("LOOKIE the actual values: ",y[samples].flatten())
        fval = self.compute_loss(obj_fwd, y[samples].flatten())
        
        #print("Training loss is: ", fval)
        
        return fval.item()

    def obj_grad(self, params, minibatch):
        fgrad = []
        x = self.itrain
        y = self.otrain
        samples = np.random.choice(len(y), minibatch, replace=False)
        obj_fwd = self.forward(x[samples, :]).flatten()
        obj_loss = self.compute_loss(obj_fwd, y[samples].flatten())
        obj_loss.backward()

        #max_norm = 0.5
        #clip_grad_norm_(self.parameters(), max_norm)
        for param in self.parameters():
            if param.grad is not None:
                # Clone to avoid modifying the original tensor
                fgrad.append(param.grad.data.clone().view(-1))

        # Manually set gradients to zero
        for param in self.parameters():
            if param.grad is not None:
                param.grad.data.zero_()

        fgrad = torch.cat(fgrad, dim=0)
        return fgrad
    
    def constraint_loss(self, dist1, dist2):
        sum = 0
        for i in range(len(dist1)):
            sum = sum + dist1[i]*torch.log(dist1[i]/dist2[i])
        return sum
    
    def jensen_shannon_divergence(self, p, q):
        m = 0.5 * (p + q)
        kl_div_p_m = F.kl_div(p.log(), m, reduction='batchmean')
        kl_div_q_m = F.kl_div(q.log(), m, reduction='batchmean')
        return 0.5 * (kl_div_p_m + kl_div_q_m)

    def get_cov(self, f_val, a):
        #print("Calculating Cov:")
        #print(len(f_val))
        #print(len(a))
        a_avg = torch.mean(a)
        sum = 0
        for i in range(len(a)):
           sum = sum + (a[i]-a_avg)*f_val[i]
        print("Covariance between Pred and race label is: ", sum**2)
        return sum**2


    def conf1(self, params, minibatch):
        #print("Reached at function constraint")
        conf_val = None
        x_train = self.itrain
        y_train = self.otrain
        x_train_raw = self.itrain_raw
        x_female = x_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_1), :]
        y_female = y_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_1)]
        x_male = x_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_0), :]
        y_male = y_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_0)]
        
        # print("Total no of males with y=0=", x_males_0_lab.shape[0])
        #self.RANDOM_SEED = np.random.randint(0,len(y_train))
        #np.random.seed(self.RANDOM_SEED)
        female_samples = np.random.choice(len(y_female), minibatch, replace=False)
        #print("Selected female samples for ConsVal1:", female_samples)
        #np.random.seed(self.RANDOM_SEED)
        male_samples = np.random.choice(len(y_male), minibatch, replace=False)
        #print("Selected male samples for ConsVal1:", male_samples)
        #samples = np.random.choice(len(y_val), minibatch, replace=False)
        #conf_val = self.forward(x_val[minibatch, :])
        cons_fwd_female = self.forward(x_female[female_samples, :]).flatten()
        cons_loss_female = self.compute_loss(cons_fwd_female, y_train[female_samples].flatten())

        cons_fwd_male = self.forward(x_male[male_samples, :]).flatten()
        cons_loss_male = self.compute_loss(cons_fwd_male, y_train[male_samples].flatten())

        cons_loss = (cons_loss_female - cons_loss_male)
        #cons_loss = cons_loss_female

        # print("Avg Loss over female samples Constraint1: ", cons_loss_female)
        # print("Avg Loss over male samples Constraint1: ", cons_loss_male)
        # print("Validation loss is: ", cons_loss)
        
        return (cons_loss.item())
    
    def conJ1(self, params, minibatch):
        #print("Reached at function constraint grad")
        cgrad = []
        x_train = self.itrain
        y_train = self.otrain
        x_train_raw = self.itrain_raw
        x_female = x_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_1), :]
        y_female = y_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_1)]
        x_male = x_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_0), :]
        y_male = y_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_0)]

        #np.random.seed(self.RANDOM_SEED)
        female_samples = np.random.choice(len(y_female), minibatch, replace=False)
        #print("Selected female samples for ConsGrad1:", female_samples)
        #np.random.seed(self.RANDOM_SEED)
        male_samples = np.random.choice(len(y_male), minibatch, replace=False)
        #print("Selected male samples for ConsGrad1:", male_samples)
        # samples = np.random.choice(len(y_val), minibatch, replace=False)
        # conf_val = self.forward(x_val[minibatch, :])
        cons_fwd_female = self.forward(x_female[female_samples, :]).flatten()
        cons_loss_female = self.compute_loss(cons_fwd_female, y_train[female_samples].flatten())

        cons_fwd_male = self.forward(x_male[male_samples, :]).flatten()
        cons_loss_male = self.compute_loss(cons_fwd_male, y_train[male_samples].flatten())
        #cons_loss = cons_loss_female
        cons_loss = (cons_loss_female - cons_loss_male)
        cons_loss.backward()
        max_norm = 0.5
        clip_grad_norm_(self.parameters(), max_norm)
        for param in self.parameters():
            if param.grad is not None:
                cgrad.append(param.grad.data.clone().view(-1))  # Clone to avoid modifying the original tensor

# Manually set gradients to zero without using optimizer.zero_grad()
        for param in self.parameters():
            if param.grad is not None:
                param.grad.data.zero_()

        
        cgrad = torch.cat(cgrad, dim=0)
        
        return cgrad

    def conf2(self, params, minibatch):
        #print("Reached at function constraint")
        conf_val = None
        x_train = self.itrain
        y_train = self.otrain
        x_train_raw = self.itrain_raw
        x_female = x_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_1), :]
        y_female = y_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_1)]
        x_male = x_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_0), :]
        y_male = y_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_0)]
        
        # print("Total no of males with y=0=", x_males_0_lab.shape[0])
        #self.RANDOM_SEED = np.random.randint(0,len(y_train))
        #np.random.seed(self.RANDOM_SEED)
        female_samples = np.random.choice(len(y_female), minibatch, replace=False)
        #print("Selected female samples for ConsVal2:", female_samples)
        #np.random.seed(self.RANDOM_SEED)
        male_samples = np.random.choice(len(y_male), minibatch, replace=False)
        #print("Selected male samples for ConsVal2:", male_samples)
        #samples = np.random.choice(len(y_val), minibatch, replace=False)
        #conf_val = self.forward(x_val[minibatch, :])
        cons_fwd_female = self.forward(x_female[female_samples, :]).flatten()
        cons_loss_female = self.compute_loss(cons_fwd_female, y_train[female_samples].flatten())

        cons_fwd_male = self.forward(x_male[male_samples, :]).flatten()
        cons_loss_male = self.compute_loss(cons_fwd_male, y_train[male_samples].flatten())

        cons_loss = (cons_loss_male - cons_loss_female)
        #cons_loss = cons_loss_female

        # print("Avg Loss over female samples Constraint2: ", cons_loss_female)
        # print("Avg Loss over male samples Constraint2: ", cons_loss_male)
        # print("Validation loss is: ", cons_loss)
        
        return (cons_loss.item())
    
    def conJ2(self, params, minibatch):
        #print("Reached at function constraint grad")
        cgrad = []
        x_train = self.itrain
        y_train = self.otrain
        x_train_raw = self.itrain_raw
        x_female = x_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_1), :]
        y_female = y_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_1)]
        x_male = x_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_0), :]
        y_male = y_train[(x_train_raw[:, GENDER_IND] == SENSITIVE_CODE_0)]
        
        #np.random.seed(self.RANDOM_SEED)
        female_samples = np.random.choice(len(y_female), minibatch, replace=False)
        #print("Selected female samples for ConsGrad2:", female_samples)
        #np.random.seed(self.RANDOM_SEED)
        male_samples = np.random.choice(len(y_male), minibatch, replace=False)
        #print("Selected male samples for ConsGrad2:", male_samples)
        # samples = np.random.choice(len(y_val), minibatch, replace=False)
        # conf_val = self.forward(x_val[minibatch, :])
        cons_fwd_female = self.forward(x_female[female_samples, :]).flatten()
        cons_loss_female = self.compute_loss(cons_fwd_female, y_train[female_samples].flatten())

        cons_fwd_male = self.forward(x_male[male_samples, :]).flatten()
        cons_loss_male = self.compute_loss(cons_fwd_male, y_train[male_samples].flatten())
        #cons_loss = cons_loss_female
        cons_loss = (cons_loss_male - cons_loss_female)
        cons_loss.backward()
        max_norm = 0.5
        clip_grad_norm_(self.parameters(), max_norm)
        for param in self.parameters():
            if param.grad is not None:
                cgrad.append(param.grad.data.clone().view(-1))  # Clone to avoid modifying the original tensor

# Manually set gradients to zero without using optimizer.zero_grad()
        for param in self.parameters():
            if param.grad is not None:
                param.grad.data.zero_()

        
        cgrad = torch.cat(cgrad, dim=0)
        
        return cgrad

In [10]:
def paramvals(maxiter, beta, rho, lamb, hess, tau, mbsz, numcon, geomp, stepdecay, gammazero, zeta, N, n, lossbound, scalef):
    params = {
        'maxiter': maxiter,  # number of iterations performed
        'beta': beta,  # trust region size
        'rho': rho,  # trust region for feasibility subproblem
        'lamb': lamb,  # weight on the subfeasibility relaxation
        'hess': hess,  # method of computing the Hessian of the QP, options include 'diag' 'lbfgs' 'fisher' 'adamdiag' 'adagraddiag'
        'tau': tau,  # parameter for the hessian
        'mbsz': mbsz,  # the standard minibatch size, used for evaluating the progress of the objective and constraint
        'numcon': numcon,  # number of constraint functions
        'geomp': geomp,  # parameter for the geometric random variable defining the number of subproblem samples
        # strategy for step decrease, options include 'dimin' 'stepwise' 'slowdimin' 'constant'
        'stepdecay': stepdecay,
        'gammazero': gammazero,  # initial stepsize
        'zeta': zeta,  # parameter associated with the stepsize iteration
        'N': N,  # Train/val sample size
        'n': n,  # Total number of parameters
        'lossbound': lossbound, #Bound on constraint loss
        'scalef': scalef #Scaling factor for constraints
    }
    return params

In [11]:
import sys
import types

def reload_package(root_module):
    package_name = root_module.__name__

    # get a reference to each loaded module
    loaded_package_modules = dict([
        (key, value) for key, value in sys.modules.items()
        if key.startswith(package_name) and isinstance(value, types.ModuleType)])

    # delete references to these loaded modules from sys.modules
    for key in loaded_package_modules:
        del sys.modules[key]

    # load each of the modules again;
    # make old modules share state with new modules
    for key in loaded_package_modules:
        print("Loading ", key)
        newmodule = __import__(key)
        oldmodule = loaded_package_modules[key]
        oldmodule.__dict__.clear()
        oldmodule.__dict__.update(newmodule.__dict__)

In [12]:
import StochasticGhost
reload_package(StochasticGhost)

Loading  StochasticGhost


The Training loop, does the following:

1) Makes 21 different training iterations.
2) makes call to the Ghost package and passes the Objective and constraints functions along with other parameters.
3) Ghost module trains the model by solving the problem as a SQP.
4) Each model is saved after training.
5) Models can be loaded for further analysis and refinement using the package model_selector in the same directory.

In [13]:
######Training loop######
loss_bound=1
trials = 21
maxiter = 200
acc_arr = []
max_acc = 0
ftrial = np.zeros((maxiter, trials))
ctrial1 = np.zeros((maxiter, trials))
ctrial2 = np.zeros((maxiter, trials))
initsaved = []
#x_train, x_val, y_train, y_val = train_test_split(in_df.values, out_df.values, test_size=0.3, random_state=42)
ip_size = x_train.shape[1]
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val = torch.tensor(y_val, dtype=torch.float32)
saved_model = []
for trial in range(trials):
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>TRIAL", trial)
    
    #print(type(X_train))
    hid_size1 = 16
    hid_size2 = 16
    op_size = 1
    #print(X_val.shape)
    #print(Y_val.shape)
    layer_sizes = [ip_size, hid_size1, hid_size2, op_size]
    
    #x_len = x_train[:, 4]
    num_trials = min(len(y_train[((x_train[:, GENDER_IND]) == SENSITIVE_CODE_1)]), len(y_train[(x_train[:, GENDER_IND] == SENSITIVE_CODE_0)]))

    #num_trials = min(len(y_val[((x_val[:, 1]) == 0)]), len(y_val[(x_val[:, 1] == 1)]))
    #num_trials = min((y_train.reshape(-1) == 0)]), len(y_train[(x_len == 1) & (y_train.reshape(-1) == 0)]))
    #num_trials = len(y_val[(y_val.reshape(-1) == 1)])
    print(num_trials)
    #print(num_trials)
    net = CustomNetwork(
        layer_sizes, X_train[:, :ip_size], Y_train, X_val[:, :ip_size], Y_val, torch.tensor(x_train))
    #print(net)
    # net.apply(net.init_weights)
    nn_parameters = list(net.parameters())
    # print(net)
    initw = [param.data for param in nn_parameters]
    # print(len(initw))
    num_param = sum(p.numel() for p in net.parameters())
    params = paramvals(maxiter=maxiter, beta=10., rho=3e-1, lamb=0.5, hess='diag', tau=1., mbsz=100,
                       numcon=2, geomp=0.2, stepdecay='dimin', gammazero=0.1, zeta=0.1, N=num_trials, n=num_param, lossbound=[loss_bound, loss_bound], scalef=[1., 1.])
    w, iterfs, itercs = StochasticGhost.StochasticGhost(
        net.obj_fun, net.obj_grad, [net.conf1, net.conf2], [net.conJ1, net.conJ2], initw, params)
    ftrial[:, trial] = iterfs
    #print("The moment of TRUTH",itercs.shape)
    ctrial1[:, trial] = itercs[:,0]
    ctrial2[:, trial] = itercs[:,1]

    #outputs = net.forward(X_val)
    #predictions = (outputs >= 0.5).float()
    #true = Y_val
    #print(predictions.flatten())
    #print(Y_val.flatten())
    #acc = len(np.where(predictions.flatten() == Y_val.flatten())[0]) / len(Y_val.flatten())
    saved_model.append(net)
    torch.save(net, 'income_models/saved_model'+str(trial)+'.pth')
    #acc_arr.append(acc)
    

>>>>>>>>>>>>>>>>>>>>>>>>>>>>TRIAL 0
6832
>>>>>step_norm<<<<<< 0.4320633806332014


/home/choudhar/anaconda3/envs/ghost/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/choudhar/anaconda3/envs/ghost/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted G to scipy.sparse.csc.csc_matrix
For best performance, build G as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/choudhar/anaconda3/envs/ghost/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(


>>>>>step_norm<<<<<< 0.33156860536751775
>>>>>step_norm<<<<<< 0.40142518621924755
>>>>>step_norm<<<<<< 1.0098585341247632
>>>>>step_norm<<<<<< 0.3599921836049872
>>>>>step_norm<<<<<< 0.512065871194822
>>>>>step_norm<<<<<< 0.5298338419903477
>>>>>step_norm<<<<<< 0.7680425385012193
>>>>>step_norm<<<<<< 0.5636422900003529
>>>>>step_norm<<<<<< 0.5106995819793574
>>>>>step_norm<<<<<< 0.6106440518726914
>>>>>step_norm<<<<<< 0.5715002497091433
>>>>>step_norm<<<<<< 0.2912798428511047
>>>>>step_norm<<<<<< 1.7184328499335122
>>>>>step_norm<<<<<< 0.1233332262666567
>>>>>step_norm<<<<<< 0.7913582096797395
>>>>>step_norm<<<<<< 0.40166685758176995
>>>>>step_norm<<<<<< 0.35038374885877566
>>>>>step_norm<<<<<< 0.31179901267878596
>>>>>step_norm<<<<<< 0.5726358597903306
>>>>>step_norm<<<<<< 0.5488789199831865
>>>>>step_norm<<<<<< 0.4560148024159863
>>>>>step_norm<<<<<< 0.3399798860304536
>>>>>step_norm<<<<<< 0.4416295347004677
>>>>>step_norm<<<<<< 0.37383696859269033
>>>>>step_norm<<<<<< 0.312288609763

In [1]:
print(">>>>>>>>>>>>>>>>>>>ACCURACY ARRAY<<<<<<<<<<<<<<<<")
#print(acc_arr)
df_ftrial = pd.DataFrame(ftrial, columns=range(1, ftrial.shape[1]+1), index=range(1, ftrial.shape[0]+1))
df_ctrial1 = pd.DataFrame(ctrial1, columns=range(1, ctrial1.shape[1]+1), index=range(1, ctrial1.shape[0]+1))
df_ctrial2 = pd.DataFrame(ctrial2, columns=range(1, ctrial2.shape[1]+1), index=range(1, ctrial2.shape[0]+1))

# Save DataFrames to CSV files
df_ftrial.to_csv('income_iters/income_ftrial_new.csv')
df_ctrial1.to_csv('income_iters/income_ctrial1_new.csv')
df_ctrial2.to_csv('income_iters/income_ctrial2_new.csv')

>>>>>>>>>>>>>>>>>>>ACCURACY ARRAY<<<<<<<<<<<<<<<<


NameError: name 'pd' is not defined